In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import  matplotlib.pyplot as plt

In [4]:
from isingHMC import IsingHMC

In [8]:
np.linspace(4,1.4,51)

array([4.   , 3.948, 3.896, 3.844, 3.792, 3.74 , 3.688, 3.636, 3.584,
       3.532, 3.48 , 3.428, 3.376, 3.324, 3.272, 3.22 , 3.168, 3.116,
       3.064, 3.012, 2.96 , 2.908, 2.856, 2.804, 2.752, 2.7  , 2.648,
       2.596, 2.544, 2.492, 2.44 , 2.388, 2.336, 2.284, 2.232, 2.18 ,
       2.128, 2.076, 2.024, 1.972, 1.92 , 1.868, 1.816, 1.764, 1.712,
       1.66 , 1.608, 1.556, 1.504, 1.452, 1.4  ])

In [6]:
L = 100

N = L**2
J = 1
beta = 0.8
h = 0
T = 2
eps = 0.1

ising = IsingHMC(J=J, L=L, beta=beta, h=h)
phi = np.random.uniform(0,1,size=N)

for beta_inv in np.linspace(4, 1.4, 5):
    ising.set_beta(1/beta_inv)
    ising.initialize(phi)
    
    [phi] = ising.run(T=1, Ncf = 101, eps = eps)

10
conf 0/101, accept: 1.00. accept pre therm: 0.00, eps: 0.1
conf 5/101, accept: 0.83. accept pre therm: 0.00, eps: 0.1
conf 10/101, accept: 0.91. accept pre therm: 0.00, eps: 0.1


KeyboardInterrupt: 

In [ ]:
[]

In [ ]:

phi = np.random.uniform(0,1,size=N)
# phi = np.ones(N)
T = 1
eps = 0.10
Nmd = int(T/eps)
print(Nmd)

Ncf = 10000
Ncorr = 5
Ntherm = 100
phi_values = []

from IPython.display import clear_output
reset = False
acc_pre = 0
for i in range(Ncf):
    phi = ising.hmc_sample(phi, eps, Nmd)
    acc = ising.acceptance()
    if i%10 == 0 and i >= Ntherm:
        phi_values.append(phi)
        if not reset:
            acc_pre = ising.acceptance()
            ising.reset_acceptance()
            reset = True
        
    clear_output(wait=True)
    print(f"conf {i}/{Ncf}, accept: {acc:.2f}. accept pre therm: {acc_pre:.2f}, eps: {eps}")
    

In [ ]:
phi_values = np.array(phi_values)

In [ ]:
def bootstrap(values, func=lambda x:np.mean(x,axis=0), K=50, verbose=False):
    Nsamples = values.shape[0]
    
    fvalues = []
    for i in range(K):
        if verbose:
            clear_output(wait=True)
            print(f"{i+1}/{K}")
        choice = np.random.randint(Nsamples, size=Nsamples)
        sample = values[choice]
        fvalues.append(func(sample))
    return np.array(fvalues)

In [ ]:
be = ising.betaEps(phi_values)
eps = bootstrap(be, func=lambda x:np.mean(x,axis=0)**2, verbose=True)
eps2 = bootstrap(be, func=lambda x:np.mean(x**2,axis=0), verbose=True)

cv = bootstrap(be, func=lambda x:np.mean(x,axis=0)**2 - np.mean(x**2, axis=0), verbose=True)
cv = ising.N*cv/ising.beta

In [ ]:
# _,bins,_ = plt.hist(eps-eps2,bins=100)
plt.hist(cv, bins=20);

In [ ]:
np.mean(cv), np.std(cv)

In [ ]:
import uncertainties as uc

print('epsilon Beta = uc.ufloat(np.mean(eps), np.std(eps)))')

In [ ]:
m = ising.magnetization(phi_values)

betaeps = ising.betaEps(phi_values)


In [ ]:
plt.plot(betaeps)

In [ ]:
ac_m = acf(m)
plt.plot(ac_m)
plt.axhline(1/np.exp(1))
np.where(ac_m < 1/np.exp(1))[0][0]

In [ ]:
def acf(x, length=100):
    return np.array([1]+[np.corrcoef(x[:-i], x[i:])[0,1]  \
        for i in range(1, length)])

# Random animation thingy

In [ ]:
import matplotlib.animation
from mpl_toolkits.axes_grid1 import make_axes_locatable

L = L
N = L**2
phi = np.zeros(N) + np.random.random(size=N)
ising = IsingHMC(J=1, L=L, beta=1)
T = 0.1
eps = 0.01
Nmd = int(T/eps)

points = np.arange(N)
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

cf = ax.contourf(phi.reshape((L,L)), 200)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title('Frame 0')

stupid_list = [phi]

def update(i):
    phi = stupid_list[0]
    stupid_list[0] = ising.hmc_sample(phi, eps, Nmd)

    vmax     = 5# np.max(phi)
    vmin     = -5 # np.min(phi)
    levels   = np.linspace(vmin, vmax, 200, endpoint = True)
    cf = ax.contourf(phi.reshape((L,L)), vmax=vmax, vmin=vmin, levels=levels)
    cax.cla()
    fig.colorbar(cf, cax=cax)
    tx.set_text('Frame {0}'.format(i))

ani = matplotlib.animation.FuncAnimation(fig, update, 100)

HTML(ani.to_jshtml())

In [ ]:
Writer = animation.writers['imagemagick']

In [ ]:
writer = Writer(fps=15, metadata=dict(artist='Halvard'), bitrate=1800)

ani.save('fluctuation.mp4', writer=writer)